# hisea use case workflow notebook

This notebook provides an example of running the hisea workflow.

## prerequisites
1. build the docker containers to download the data following instructions on <https://github.com/c-scale-community/use-case-hisea/tree/main/scripts/download>
2. build the docker container to preprocess the data following instructions on ... TODO

## download ERA5 data
Notes:
* path to my .cdsapirc file\
`/Users/backeb/.cdsapirc`
* path to where I am downloading the data\
`~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718`

* Other input parameters:\
`--longitude_min 22.5`\
`--longitude_max 24.5`\
`--latitude_min 36.5`\
`--latitude_max 38.5`\
`--date_min '2022-04-01'`\
`--date_max '2022-04-05'`

In [ ]:
!docker run -v /Users/backeb/.cdsapirc:/root/.cdsapirc -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data download-input python download_era5.py --longitude_min 22.5 --longitude_max 24.5 --latitude_min 36.5 --latitude_max 38.5 --date_min '2022-04-01' --date_max '2022-04-05'

## download CMEMS data
Notes:
* path to where I am downloading the data\
`~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718`
* Other input parameters:\
`--username bbackeberg`\
`--password iaTmwJ7D`\
`--longitude_min 22.5`\
`--longitude_max 24.5`\
`--latitude_min 36.5`\
`--latitude_max 38.5`\
`--date_min '2022-04-01'`\
`--date_max '2022-04-05'`

In [ ]:
!docker run -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data download-input python download_cmems_physics.py --username bbackeberg --password iaTmwJ7D --longitude_min 22.5 --longitude_max 24.5 --latitude_min 36.5 --latitude_max 38.5 --date_min '2022-04-01' --date_max '2022-04-05'
# TODO add line to download biogeochemistry

In [ ]:
ls -ltr ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718

## preprocess with hydrolib
I need to create the following files:
* salinity_east2.bc
* salinity_south2.bc
* steric_east2.bc
* steric_south2.bc
* temperature_east2.bc
* temperature_south2.bc
* tide_east2.bc
* tide_south2.bc
* uxuy_east2.bc

and I have the following files:
* DFMWAQ_east2_tmp.ext
* DFMWAQ_south2_tmp.ext
* boundary1_waq.ext
* east2.ext
* east2.pli
* south2.ext
* south2.pli
* wind_heat_waq.ext

## load .nc into memory

In [4]:
import xarray as xr
# xarray dependencies:
#   conda install xarray
#   conda install netcdf4
#   conda install -c conda-forge h5netcdf
#   conda install scipy
dname = '/Users/backeb/OneDrive - Stichting Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/'
ds = xr.open_dataset(dname+'cmems_thetao_bottomT_so_zos_uo_vo.nc')




In [5]:
ds

<xarray.Dataset>
Dimensions:    (depth: 50, latitude: 25, time: 5, longitude: 25)
Coordinates:
  * depth      (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * latitude   (latitude) float32 36.5 36.58 36.67 36.75 ... 38.33 38.42 38.5
  * time       (time) datetime64[ns] 2022-04-01T12:00:00 ... 2022-04-05T12:00:00
  * longitude  (longitude) float32 22.5 22.58 22.67 22.75 ... 24.33 24.42 24.5
Data variables:
    vo         (time, depth, latitude, longitude) float32 ...
    thetao     (time, depth, latitude, longitude) float32 ...
    uo         (time, depth, latitude, longitude) float32 ...
    bottomT    (time, latitude, longitude) float32 ...
    so         (time, depth, latitude, longitude) float32 ...
    zos        (time, latitude, longitude) float32 ...
Attributes: (12/25)
    title:                              daily mean fields from Global Ocean P...
    easting:                            longitude
    northing:                           latitude
    history:                            2022/07/18 00:05:49 MERCATOR OCEAN Ne...
    source:                             MERCATOR PSY4QV3R1
    institution:                        MERCATOR OCEAN
    ...                                 ...
    FROM_ORIGINAL_FILE__longitude_max:  179.91667
    FROM_ORIGINAL_FILE__latitude_min:   -80.0
    FROM_ORIGINAL_FILE__latitude_max:   90.0
    z_min:                              0.494025
    z_max:                              5727.917
    _CoordSysBuilder:                   ucar.nc2.dataset.conv.CF1Convention

## preprocessing with coastserv

Notes
* to build the preprocessing container I did the following
    * in `use-case-hisea/scripts/preprocessing/tide_physical_chemical/` do:\
    `docker build --tag preprocessing .`
* to run the preprocessing in the docker container for `boundary.py` do:\
`docker run -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data/input -v /Users/backeb/Documents/REPOS/use-case-hisea/fm_model:/data/model -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/preprocess_output:/data/output preprocessing boundary.py --interp true --simultaneous true --steric true --input /data/input --model /data/model --output /data/output`\
the above has the following input parameters:
    * the directory from which to get the downloaded CMEMS data:\
    `~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718`
    * the directory where all the model files are:\
    `/Users/backeb/Documents/REPOS/use-case-hisea/fm_model`
    * the directory where to dump the data after the preprocessing step:\
   `~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/preprocess_output`\

In [14]:
!docker run -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718:/data/input -v /Users/backeb/Documents/REPOS/use-case-hisea/fm_model:/data/model -v ~/OneDrive\ -\ Stichting\ Deltares/Desktop/Project-C-SCALE/0-HiSea_devs/tests/20220718/preprocess_output:/data/output preprocessing boundary.py --interp true --simultaneous true --steric true --input /data/input --model /data/model --output /data/output

ERROR conda.cli.main_run:execute(41): `conda run python boundary.py --interp true --simultaneous true --steric true --input /data/input --model /data/model --output /data/output` failed. (See above for error)
 
Model reference date is: 2015-01-01 00:00:00
 
running with options interp = True, simultaneous = True, steric = true
files for phyc: 0
files for phyc: 0
files for phyc: 0
files for phyc: 0
files for o2: 0
files for no3: 0
files for po4: 0
files for so: 1
files for thetao: 1
files for uo: 1
files for zos: 1
files for phyc: 0
files for phyc: 0
files for si: 0
suspected missing subs:
Opal
PON1
POP1
POC1
OXY
NO3
PO4
Green
Diat
Si
subs:
['Opal', 'PON1', 'POP1', 'POC1', 'OXY', 'NO3', 'PO4', 'salinity', 'temperature', 'uxuy', 'steric', 'Green', 'Diat', 'Si']
# of files available:
[0 0 0 0 0 0 0 1 1 1 1 0 0 0]
['salinity', 'temperature', 'uxuy', 'steric'] newsublist
substance: salinity
reading data...
['so'] printing the csub value
reading data file cmems_thetao_bottomT_so_zos_uo_vo.nc